In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
# Find string between two substrings
# https://stackoverflow.com/questions/3368969/find-string-between-two-substrings
import re
import time

start_time = time.time()

html = urlopen("http://www.chemfaces.com/sitemap.xml")
chemfaces = BeautifulSoup(html)

loc_list = []
loc = chemfaces.findAll("loc")
for l in loc:
    loc_list.append(l.get_text())

natural = [x for x in loc_list if 'natural' in x]

# Initializing Multiple Lists/Line
# https://stackoverflow.com/questions/2402646/python-initializing-multiple-lists-line
t, cat, cas, p, s, d = ([] for i in range(6))

for i in natural:
    html = urlopen(i)
    content = BeautifulSoup(html)
    try:
        title = content.find("div", {"class": "pi_title"})
        title_text = title.get_text()
        cfn = content.find("script", {"type":"text/javascript"}).get_text()
        result_cat = re.search('CFN(.*).html', cfn)
        cat.append("CFN" + result_cat.group(1))
        c = content.find("td", {"width": "224"}).get_text()
        source = content.find("td", {"class":"p_c_tbl_2"}, {"width":"1040"}).get_text()
        result_p = re.search('The (.*) of', source)
        result_s = re.search('of (.*)', source)
        p.append(result_p.group(1))
        s.append(result_s.group(1))
        description = content.find("td", {"class": "bd_tbl_2"}).get_text()
    except AttributeError as e:
        t.append("None")
        cat.append("None")
        cas.append("None")
        p.append("None")
        s.append("None")
        d.append("None")
    else:
        t.append(title_text)
        cas.append(c)
        d.append(description)

# Remove characters
# https://stackoverflow.com/questions/8282553/removing-character-in-list-of-strings
T = [i.strip('\n') for i in t]
t = pd.Series(T, name='Name')
cas = pd.Series(cas, name='CAS')
cat = pd.Series(cat, name='Catalog No.')
p = pd.Series(p, name='Part')
s = pd.Series(s, name='Source')
d = pd.Series(d, name='Description')
total = pd.concat([t, cat, cas, p, s, d], axis=1)

total.to_excel("Natural.xlsx")

print("%s seconds" % (time.time() - start_time))

15.452223539352417 seconds
